**Step 1: **create a data folder in collab and upload Anil cv and mailer.docx file

In [ ]:
# Install the OpenAI library to interact with OpenAI's GPT models.
!pip install openai==0.28

# Install FAISS, a library for efficient similarity search and clustering of dense vectors.
!pip install faiss-cpu

# Install the Fitz library, which is part of PyMuPDF, used for working with PDF documents.
!pip install fitz

# Install Pandas, a powerful data manipulation and analysis library for Python.
!pip install pandas

# Install NumPy, a library for numerical computations in Python.
!pip install numpy

# Install PyMuPDF, a library for working with PDF documents.
!pip install PyMuPDF

# Install python-docx, a library for creating, modifying, and extracting information from Microsoft Word (.docx) documents.
!pip install python-docx


Summary
Embedding Documents: The embed_documents function converts each document into a vector embedding using OpenAI's embedding model.

Indexing with FAISS: The index_documents function creates a FAISS index and adds the embeddings to it, allowing for efficient similarity searches.


Querying the Index: The query_index function converts a query into an embedding and searches the FAISS index to find the most similar document embeddings, filtering based on a similarity threshold.


Generating Answers: The generate_answers function uses the context of the most relevant documents to generate a response with GPT-3.5-turbo.

In [ ]:
import openai
import os
import fitz  # PyMuPDF
import docx
import pandas as pd
import numpy as np
import faiss

# Set OpenAI API key
openai.api_key = "sk-proj-SLD1kkrCM3KhATIGjURlT3BlbkFJk75IwDuvAHX10HbHz8iq"

def load_documents(folder_path):
    # Create an empty list to hold the documents
    documents = []

    # Loop through each file in the specified folder
    for filename in os.listdir(folder_path):
        # Get the full path of the file
        file_path = os.path.join(folder_path, filename)

        # Check if the file is a PDF
        if filename.endswith('.pdf'):
            # Load the PDF file and add its content to the documents list
            documents.append(load_pdf(file_path))

        # Check if the file is a Word document
        elif filename.endswith('.docx'):
            # Load the Word document and add its content to the documents list
            documents.append(load_docx(file_path))

        # Check if the file is a text file
        elif filename.endswith('.txt'):
            # Load the text file and add its content to the documents list
            documents.append(load_txt(file_path))

        # Check if the file is a CSV file
        elif filename.endswith('.csv'):
            # Load the CSV file and add its content to the documents list
            documents.append(load_csv(file_path))

    # Return the list of documents
    return documents

def load_pdf(file_path):
    # Initialize an empty string to store the text
    text = ""
    # Open the PDF file
    doc = fitz.open(file_path)
    # Loop through each page in the PDF
    for page in doc:
        # Extract text from the page and add it to the text string
        text += page.get_text()
    # Return the combined text from all pages
    return text

def load_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)
    # Extract text from each paragraph in the document
    text = [paragraph.text for paragraph in doc.paragraphs]
    # Join all paragraphs into a single string separated by newlines and return it
    return "\n".join(text)

def load_txt(file_path):
    # Open the text file in read mode
    with open(file_path, 'r') as file:
        # Read the entire content of the file and return it
        return file.read()

def load_csv(file_path):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    # Convert the DataFrame to a string representation and return it
    return df.to_string()

def embed_documents(documents, model='text-embedding-ada-002'):
    # Initialize an empty list to store the embeddings
    embeddings = []
    # Loop through each document
    for doc in documents:
        # Create an embedding for the document using the specified model
        response = openai.Embedding.create(input=doc, model=model)
        # Add the embedding to the embeddings list
        embeddings.append(response['data'][0]['embedding'])
    # Return the list of embeddings
    return embeddings

def index_documents(embeddings):
    # Get the dimensionality of the embeddings (length of the first embedding)
    dimension = len(embeddings[0])
    # Create a FAISS index for storing the embeddings, using L2 (Euclidean) distance
    index = faiss.IndexFlatL2(dimension)
    # Add the embeddings to the FAISS index
    index.add(np.array(embeddings))
    # Return the FAISS index
    return index

def query_index(query, index, documents, model='text-embedding-ada-002', top_k=5):
    # Create an embedding for the query using the specified model
    response = openai.Embedding.create(input=query, model=model)
    query_embedding = response['data'][0]['embedding']

    # Search the FAISS index for the top_k most similar embeddings to the query embedding
    D, I = index.search(np.array([query_embedding]), top_k)
    # Retrieve the documents corresponding to the top_k most similar embeddings
    results = [documents[i] for i in I[0]]

    # Return the list of matching documents
    return results

def generate_answers(query, context, model='gpt-3.5-turbo'):
    if not context.strip():
        return "I can only answer questions based on the documents provided in the /content/data folder. Please ask a question related to those documents."

    messages = [
        {"role": "system", "content": "You are a helpful assistant.you can only answer questions based on the documents provided in the /content/data folder. Please ask a question related to those documents. "},
        {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}\nAnswer:"}
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=150
    )
    return response.choices[0].message['content'].strip()


def main():
    # Specify the path to the folder containing the documents
    folder_path = '/content/data'
    # Load the documents from the folder
    documents = load_documents(folder_path)
    # Create embeddings for the loaded documents
    embeddings = embed_documents(documents)
    # Index the embeddings using FAISS
    index = index_documents(embeddings)

    while True:
        # Prompt the user to enter a question
        query = input("Enter your question: ")
        # Check if the user wants to exit the loop
        if query.lower() in ["exit", "quit"]:
            break
        # Search the index for documents relevant to the query
        results = query_index(query, index, documents)
        # Combine the text of the relevant documents into a single string
        context = " ".join(results)
        # Generate an answer based on the query and the context
        answer = generate_answers(query, context)
        # Print the answer
        print(f"Answer: {answer}")

if __name__ == "__main__":
    # Run the main function when the script is executed
    main()


Answer: Siva Y's salary is 1 crore Indian rupees per annum.
Answer: Yes, Siva's salary is 12 lakhs Indian rupees per annum.


who is anil bidari

when is the webinar happening?

what are cloud skills of anil bidari?